In [55]:
import sys
sys.executable

'/users/tr.xiaow/anaconda3/envs/ceqe/bin/python3'

In [122]:
import pandas
pd.set_option("display.max_colwidth",500)

In [18]:
# conda install -c pytorch torchvision cudatoolkit=10.1 pytorch
# pip install transformers==2.3.0
# pip install ipywidgets
# pip install krovetz

In [56]:
import pyterrier as pt
import os
os.environ["JAVA_HOME"] = "/local/trmaster/opt/jdk-11.0.6/"
if not pt.started():
    pt.init(mem=8000, version='snapshot', tqdm='notebook', 
            boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"]
           )

100% [..................................................] 103577339 / 103577339Done
PyTerrier 0.6.0 has loaded Terrier 5.6 (built by jitpack on 2021-09-17 14:44)


No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [57]:
from pyterrier.measures import *
# import onir_pt as onir

In [58]:
pt.ApplicationSetup.setProperty("termpipelines", "Stopwords,PorterStemmer")

INDEX_DIR="/users/tr.xiaow/Indices/disk45_text"
import os
if os.path.exists(os.path.join(INDEX_DIR, "data.properties")):
    indexref = pt.IndexRef.of(os.path.join(INDEX_DIR, "data.properties"))
index = pt.IndexFactory.of(indexref)
print(index.getCollectionStatistics().toString())
print(index.getMetaIndex().getKeys())



DPH = pt.BatchRetrieve(index, wmodel="DPH", properties={"termpipelines" : "Stopwords,PorterStemmer"}, verbose=True, metadata=["docno", "body", "title"])

DPH_QE=pt.BatchRetrieve(index, wmodel="DPH", controls={"qe":"on", "qemodel" : "Bo1"},
                                    properties={"termpipelines": "Stopwords,PorterStemmer"},
                                    verbose=True, metadata=["docno", "body", "title"])


BM25 = pt.BatchRetrieve(index, wmodel="BM25", properties={"termpipelines" : "Stopwords,PorterStemmer"},controls={"c" : 0.7,"bm25.k_1":1.6}, verbose=True, metadata=["docno", "body", "title"])


eval_metrics = ["map", "num_rel_ret", "ndcg", "ndcg_cut_10", "ndcg_cut_20","recip_rank","recall_100","recall_1000"]

22:33:34.831 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 646.2 MiB of memory would be required.
Number of documents: 528155
Number of terms: 738439
Number of postings: 90931618
Number of fields: 0
Number of tokens: 156321446
Field names: []
Positions:   false

['docno', 'title', 'body']


In [114]:
RM3 = pt.rewrite.RM3(index)

In [59]:
# RobustNoReg
topicsRobustDESC=pt.get_dataset("trec-robust-2004").get_topics(whitelist=["DESC"], blacklist=["TITLE","NARR"])
topicsRobustTT=pt.get_dataset("trec-robust-2004").get_topics(whitelist=["TITLE"], blacklist=["DESC","NARR"])
qrelsRobust=pt.get_dataset("trec-robust-2004").get_qrels()

In [115]:
(BM25>>RM3)(topicsRobustTT.head(1))

BR(BM25):   0%|          | 0/1 [00:00<?, ?q/s]

,qid,query_0,query
0,301,international organized crime,applypipeline:off intern^0.200000018 russian^0...


# pyterrier_CEQE

In [72]:
import os
import torch
import pandas as pd
import pyterrier as pt
from pyterrier import tqdm
from pyterrier.transformer import TransformerBase
from typing import Union, Tuple
import random

import sys
import operator
import math
import numpy as np
import argparse
from pathlib import Path
import krovetz
import pprint

import extract_features as ex_f
import main.tokens_to_terms as tokens_to_terms
from graph import *
from loader import *
from utils import writer
from context_rm import get_similarity, query_vs_per_doc_context_rm, queryTerm_vs_per_doc_context_rm

In [109]:
# class ceqe:
from pyterrier.transformer import TransformerBase
import pandas as pd
def inputFeature_to_dict(feature):
    feature_dict = {'input_ids': feature.input_ids, 'input_mask': feature.input_mask,
                    'segment_ids': feature.segment_ids, 'tokens': feature.input_tokens, 'id': feature.guid}
    return feature_dict
stopwords_file = "./data/rmstop.txt"
temp = []
with open(stopwords_file) as f:
    for line in f:
        temp.append(line.strip())
stopwords = set(temp)
# args.stopwords = stopwords

class ceqe(TransformerBase):
    def __init__(self, num_fb_docs:int=2, num_exp_terms: int=20, method = "centroid",verbose : bool = True):
        self.num_fb_docs = num_fb_docs
        self.num_exp_terms = num_exp_terms
        self.verbose = verbose
        self.method = method
        self.tokenizer = "bert-base-uncased"
        self.model_name_or_path = "bert-base-uncased"
        self.model_config_or_path = "bert-base-uncased"
        self.tokenizer = "bert-base-uncased"
        self.state_dict=None #"In case you want to initiate the BERT model with the weights of a fine-tuned BERT model, you need to give the weight pickle file as the state_dict.")
        self.max_seq_len = 128
        self.batch_size=100
#         self.output_dir
        self.stopwords_file = "./data/rmstop.txt"#TODO: replace with pyterrier's stopwords
        
#         self.query_file ==>REPLACE
#         self.prf_docs_path==>REPLACE
#         self.run_file ==> topics_and_res==> REPLACE
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  
        
        self.output_hidden_states = True
        self.output_attentions = False
        astate_dict = None
        self.model_config = (self.model_name_or_path, self.model_config_or_path, device, self.output_hidden_states, self.output_attentions, self.state_dict)
        # using the second to last layer of the BERT model for representing tokens
        self.pooling_layer = [-2]
        self.attention_pooling = PoolingStrategy.NONE
  
        
        
        
    def inputFeature_to_dict(feature):
        feature_dict = {'input_ids': feature.input_ids, 'input_mask': feature.input_mask,
                        'segment_ids': feature.segment_ids, 'tokens': feature.input_tokens, 'id': feature.guid}

        return feature_dict


    def get_features(self, input_id_text, chunk):
        if self.state_dict:
            state_dict = torch.load(open(self.state_dict, 'rb'))
            print('has state_dict')
        else:
            state_dict = None
        tokenizer = BertTokenizer.from_pretrained(self.tokenizer, state_dict=state_dict)
        processor = ex_f.GetEmbeddingProcessor()
        examples = processor.get_input_examples(input_id_text)
        features = []
        for example in examples:
            example_for_processing = (example, self.max_seq_len, tokenizer, chunk)
            features.append(ex_f.convert_example_to_feature(example_for_processing))

        if chunk:
            features_flattened = []
            for i in range(len(features)):
                features_flattened.extend(features[i])
            features = features_flattened

        features_dict = [inputFeature_to_dict(f) for f in features]
        return features_dict, tokenizer

    

    def get_embedding(self,features, graph_bert):
        # Getting the layers and write down
        batch_size = self.batch_size
        batch_count = math.ceil(len(features) / batch_size)
        instances_embedding_matrix = []
        for i in range(batch_count):
            embedding_matrix = graph_bert.get_embedding_matrix(features[i * batch_size: (i + 1) * batch_size])
            instances_embedding_matrix.append(embedding_matrix)
        return torch.cat(instances_embedding_matrix, 0)


    def remove_zero_tokens(self,embedding_matrix):
        embedding_matrix_non_zero = []
        for i in range(embedding_matrix.shape[0]):
            # embedding matrix for one instance
            embedding_matrix_i = embedding_matrix[i]
            embedding_matrix_i_non_zero = []
            for k in embedding_matrix_i:
                if np.array_equal(k, np.zeros(embedding_matrix_i.shape[1])):
                    break
                embedding_matrix_i_non_zero.append(k)
            embedding_matrix_non_zero.append(np.array(embedding_matrix_i_non_zero))
        embedding_matrix_non_zero = np.array(embedding_matrix_non_zero)
        return embedding_matrix_non_zero


    def split_layers(self):
        return


    def get_info_structure(self, features, embeddings, pooling):
        info_structure = []
        if pooling == PoolingStrategy.REDUCE_MEAN:
            for i in range(len(features)):
                info_structure.append(
                    {'id': features[i]['id'], 'embedding': embeddings[i]})
        elif pooling == PoolingStrategy.NONE:
            for i in range(len(features)):
                info_structure.append(
                    {'id': features[i]['id'], 'embedding': embeddings[i], 'tokens': features[i]['tokens']})

        return info_structure


    def context_rm_query(self, query_info, list_prf_docs_tokens_info, ret_result, query):
        # only one query, prf_docs_tokens_info is for the only query
        q_id = query_info['id']
        dict_prf_docs_tokens_info = {'id': [], 'tokens': [], 'embedding': []}
        for d in range(len(list_prf_docs_tokens_info)):
            dict_prf_docs_tokens_info['id'].append(list_prf_docs_tokens_info[d]['id'])
            dict_prf_docs_tokens_info['tokens'].append(list_prf_docs_tokens_info[d]['tokens'])
            dict_prf_docs_tokens_info['embedding'].append(list_prf_docs_tokens_info[d]['embedding'])

        docs_terms, docs_terms_embeds = tokens_to_terms.get_terms(dict_prf_docs_tokens_info)
        docs_terms_embeds_pooled_np = tokens_to_terms.get_terms_embeds_pooled(docs_terms, docs_terms_embeds)

        prf_docs_terms_info = {'id': [], 'terms': [], 'embedding': []}
        for d in range(len(dict_prf_docs_tokens_info['id'])):
            prf_docs_terms_info['id'].append(dict_prf_docs_tokens_info['id'][d])
            prf_docs_terms_info['terms'].append(docs_terms[d])
            prf_docs_terms_info['embedding'].append(docs_terms_embeds_pooled_np[d])

        similarity = get_similarity.get_similarity_query_and_docTerms_CAR(query_info, prf_docs_terms_info)
       

        docsTerms_per_queryTerms_normalized_score = query_vs_per_doc_context_rm.get_docsTerms_per_query_normalized_score(
            similarity)

        
        exp_terms_score_mul_doc_prob = query_vs_per_doc_context_rm.get_exp_terms_score_mul_doc_prob(
            docsTerms_per_queryTerms_normalized_score,
            ret_result[q_id])
        unique_expansion_terms = self.get_unique_expansion_terms(exp_terms_score_mul_doc_prob, query)
        return unique_expansion_terms


    def context_rm_queryTerm(self,query_token_info, list_prf_docs_tokens_info, ret_result, query, method_type=None):
        # only one query, list_prf_docs_tokens_info is for the only query
        q_id = query_token_info['id']
        queries_token_info = {'id': [], 'tokens': [], 'embedding': []}
        queries_token_info['id'].append(query_token_info['id'])
        queries_token_info['tokens'].append(query_token_info['tokens'])
        queries_token_info['embedding'].append(query_token_info['embedding'])
        query_terms, query_terms_embeds = tokens_to_terms.get_terms(queries_token_info)
        query_terms_embeds_pooled_np = tokens_to_terms.get_terms_embeds_pooled(query_terms, query_terms_embeds)
 
        query_info = {}
        query_info['id'] = query_token_info['id']
        query_info['terms'] = query_terms[0]
        query_info['embedding'] = query_terms_embeds_pooled_np[0]

        dict_prf_docs_tokens_info = {'id': [], 'tokens': [], 'embedding': []}
        for d in range(len(list_prf_docs_tokens_info)):
            dict_prf_docs_tokens_info['id'].append(list_prf_docs_tokens_info[d]['id'])
            dict_prf_docs_tokens_info['tokens'].append(list_prf_docs_tokens_info[d]['tokens'])
            dict_prf_docs_tokens_info['embedding'].append(list_prf_docs_tokens_info[d]['embedding'])

        # group tokens to terms
        docs_terms, docs_terms_embeds = tokens_to_terms.get_terms(dict_prf_docs_tokens_info)
        docs_terms_embeds_pooled_np = tokens_to_terms.get_terms_embeds_pooled(docs_terms, docs_terms_embeds)
 
        prf_docs_terms_info = {'id': [], 'terms': [], 'embedding': []}
        for d in range(len(dict_prf_docs_tokens_info['id'])):
            prf_docs_terms_info['id'].append(dict_prf_docs_tokens_info['id'][d])
            prf_docs_terms_info['terms'].append(docs_terms[d])
            prf_docs_terms_info['embedding'].append(docs_terms_embeds_pooled_np[d])

        similarity = get_similarity.get_similarity_queryTerms_and_docsTerms_CAR(query_info, prf_docs_terms_info)
        # pprint.pprint(similarity)

        docsTerms_per_queryTerms_normalized_score = queryTerm_vs_per_doc_context_rm.get_docsTerms_per_queryTerms_normalized_score(
            similarity)
    
        if method_type == "mul":
            final_scores_mul = queryTerm_vs_per_doc_context_rm.get_docsTerms_per_query_normalized_score_mul_pool(
                docsTerms_per_queryTerms_normalized_score)
            exp_terms_score_mul_doc_prob_mul = queryTerm_vs_per_doc_context_rm.get_exp_terms_score_mul_doc_prob(
                final_scores_mul, ret_result[q_id])

            unique_expansion_terms = self.get_unique_expansion_terms(exp_terms_score_mul_doc_prob_mul, query)
            return unique_expansion_terms
                                                                                  
        elif method_type == "max":
            final_scores_max = queryTerm_vs_per_doc_context_rm.get_docsTerms_per_query_normalized_score_max_pool(
                docsTerms_per_queryTerms_normalized_score)
            exp_terms_score_mul_doc_prob_max = queryTerm_vs_per_doc_context_rm.get_exp_terms_score_mul_doc_prob(
                final_scores_max, ret_result[q_id])
            unique_expansion_terms = self.get_unique_expansion_terms(exp_terms_score_mul_doc_prob_max, query)
            return unique_expansion_terms
                                                                                    
    
    def get_unique_expansion_terms(self,exp_terms_with_score, query):
        ks = krovetz.PyKrovetzStemmer()

        new_queries = ""
        q_id = query[0][0]
        q_text = query[0][1]
        # bow_q_text = set(q_text.split(' '))
        bow_q_text = set([ks.stem(q_t) for q_t in set(q_text.split(' '))])

        most_sim = sorted(exp_terms_with_score.items(), key=operator.itemgetter(1), reverse=True)
        unique_exp_terms = []
        for t in most_sim:
            # add topk
            if len(unique_exp_terms) < self.num_exp_terms:
                # eliminate stopwords
                if t[0] not in stopwords and t[0] != '[CLS]' and t[0] != '[SEP]':
                    if ks.stem(t[0]) not in bow_q_text:
                        new_queries+=t[0]+"^"+str(t[1])+" "
            else:
                break
        return new_queries

    
    
    def transform_query(self, topic_and_res: pd.DataFrame) ->pd.DataFrame:
#         print(topic_and_res.columns)
        # generate the input query format
        first_row = topic_and_res.iloc[0]
        q_text = first_row.query
        q_id = first_row.qid
        query = [(q_id, q_text)]
#         print("QUERY is",query)
        # generate the initial retrieval run for the current query
        ret_result ={}
        for i in range (len(topic_and_res)):
            q = topic_and_res.iloc[i].qid
            d = topic_and_res.iloc[i].docno
            ret_result.setdefault(q, {})
            score = topic_and_res.iloc[i].score
            ret_result[q][d] = score
        #generate the prf_docs format
        prf_docs = []
        for i in range(self.num_fb_docs):
            prf_docs.append((topic_and_res.iloc[i].docno, topic_and_res.iloc[i].body))        
        
        query_features, tokenizer = self.get_features(query, chunk=False)
        prf_docs_features, tokenizer = self.get_features(prf_docs, chunk=True)
        
        
        graph_pooling_MEAN = Graph(self.model_config, self.pooling_layer, PoolingStrategy.REDUCE_MEAN, self.attention_pooling)
        graph_pooling_MEAN.initiate_model()

        query_embedding_matrix = self.get_embedding(query_features, graph_pooling_MEAN)
        query_embedding_matrix = query_embedding_matrix.detach().cpu().numpy()
        query_info = self.get_info_structure(query_features, np.array(query_embedding_matrix), PoolingStrategy.REDUCE_MEAN)[0]# only one query


        graph_pooling_NONE = Graph(self.model_config, self.pooling_layer, PoolingStrategy.NONE, self.attention_pooling)
        graph_pooling_NONE.initiate_model()

        prf_docs_tokens_embedding_matrix = self.get_embedding(prf_docs_features, graph_pooling_NONE)
        prf_docs_tokens_embedding_matrix = prf_docs_tokens_embedding_matrix.detach().cpu().numpy()
        prf_docs_tokens_embedding_matrix_non_zero = self.remove_zero_tokens(np.array(prf_docs_tokens_embedding_matrix))
        prf_docs_tokens_info = self.get_info_structure(prf_docs_features, prf_docs_tokens_embedding_matrix_non_zero,
                                                  PoolingStrategy.NONE)
    
        query_tokens_embedding_matrix = self.get_embedding(query_features, graph_pooling_NONE)
        query_tokens_embedding_matrix = query_tokens_embedding_matrix.detach().cpu().numpy()
        query_tokens_embedding_matrix_non_zero = self.remove_zero_tokens(np.array(query_tokens_embedding_matrix))
        query_tokens_info = self.get_info_structure(query_features, query_tokens_embedding_matrix_non_zero,
                                               PoolingStrategy.NONE)[0]  # only one query
        
       # select the expansion terms according to the similarity score(three methods)
#         print("METHOD",self.method)
        if self.method =="centroid":
            weighted_exp_terms = self.context_rm_query(query_info, prf_docs_tokens_info, ret_result, query)
        elif self.method =="max":
            weighted_exp_terms = self.context_rm_queryTerm(query_tokens_info, prf_docs_tokens_info, ret_res, query, method_type="max")
        elif self.method == "mul":
            weighted_exp_terms = self.context_rm_queryTerm(query_tokens_info, prf_docs_tokens_info, ret_res, query, method_type="mul")
        
        else:
            raise ValueError("Must specify a method: max, mul or centroid")

        rtr = pd.DataFrame([[first_row.qid,
                            first_row.docno,
                            first_row.query,
                            weighted_exp_terms]
                            ], columns=["qid","docno", "query", "ceqe_expanded_query"])
        return rtr
    
    def transform(self, topics_and_docs : pd.DataFrame) -> pd.DataFrame:
        rtr = []
#         print("transform cols:",topics_and_docs.columns)
        for qid, res in topics_and_docs.groupby("qid"):
            new_query_df = self.transform_query(res)    
            rtr.append(new_query_df)
        return pd.concat(rtr)
    

In [128]:
(BM25>>ceqe( num_fb_docs=2, num_exp_terms=20, method ="centroid",verbose=True)\
 >>pt.apply.query_0(lambda row: row["query"])\
 >>pt.apply.query(lambda row: "applypipeline:off "+row["ceqe_expanded_query"]) \
 >>pt.apply.query_1(drop=True)\
 >>pt.apply.ceqe_expanded_query(drop=True)>>pt.apply.docno(drop=True)
 )(topicsRobustTT.head(1))

BR(BM25):   0%|          | 0/1 [00:00<?, ?q/s]

/tmp/ipykernel_7400/3961208512.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  embedding_matrix_non_zero = np.array(embedding_matrix_non_zero)
/users/tr.xiaow/ceqe-release/main/tokens_to_terms.py:56: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(queries_terms_embeds_pooled)


,qid,query_0,query
0,301,international organized crime,applypipeline:off criminal^0.0017062496743867252 affairs^0.0011665247657870739 organs^0.0008983014181774731 internal^0.0008755314680525699 russian^0.000789959973314172 economic^0.0007703573750240684 groups^0.0006595210159383628 counterintelligence^0.0006267200155651165 measures^0.000556643955467908 banditry^0.0005554692128284883 activity^0.000522475248569999 serious^0.0005198452845253485 investigations^0.0004935254982725303 report^0.00048782945361054144 shall^0.0004792792344895164 financial^...


In [129]:
(BM25>>ceqe( num_fb_docs=2, num_exp_terms=20, method ="centroid",verbose=True)\
 >>pt.apply.query_0(lambda row: row["query"])\
 >>pt.apply.query(lambda row: "applypipeline:off "+row["ceqe_expanded_query"]) \
 >>pt.apply.query_1(drop=True)\
 >>pt.apply.ceqe_expanded_query(drop=True)>>pt.apply.docno(drop=True)>>BM25
 )(topicsRobustTT.head(1))

BR(BM25):   0%|          | 0/1 [00:00<?, ?q/s]

/tmp/ipykernel_7400/3961208512.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  embedding_matrix_non_zero = np.array(embedding_matrix_non_zero)
/users/tr.xiaow/ceqe-release/main/tokens_to_terms.py:56: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(queries_terms_embeds_pooled)


BR(BM25):   0%|          | 0/1 [00:00<?, ?q/s]

23:19:25.579 [main] WARN org.terrier.matching.PostingListManager - A negative requirement was found for the 243-th query term (05), which was past the maximum supported 64


,qid,docid,docno,body,title,rank,score,query_0,query
0,301,23985,FBIS3-23986,"""jpusr023___94049"" FBIS-USR-94-023 Document Type:JPRS Document Title: FBIS Report: Central Eurasia 10 March 1994 RUSSIA ECONOMIC AND SOCIAL AFFAIRS Seminar on Criminology Held 1990-1993 Crime Figures, Pessimistic Prognosis for Future 944E0470B Moscow DELOVOY MIR in Russian No 14, 24-30 Jan 94 p 20 944E0470B Moscow DELOVOY MIR Language: Russian Article Type:CSO [Unattributed report: ""The Forecast Is For Foul Weather (From the Theses Prepare...",,0,9.902296,international organized crime,applypipeline:off criminal^0.0017062496743867252 affairs^0.0011665247657870739 organs^0.0008983014181774731 internal^0.0008755314680525699 russian^0.000789959973314172 economic^0.0007703573750240684 groups^0.0006595210159383628 counterintelligence^0.0006267200155651165 measures^0.000556643955467908 banditry^0.0005554692128284883 activity^0.000522475248569999 serious^0.0005198452845253485 investigations^0.0004935254982725303 report^0.00048782945361054144 shall^0.0004792792344895164 financial^...
1,301,23984,FBIS3-23985,"""jpusr023___94048"" FBIS-USR-94-023 Document Type:JPRS Document Title: FBIS Report: Central Eurasia 10 March 1994 RUSSIA ECONOMIC AND SOCIAL AFFAIRS Seminar on Criminology Held Introduction to Criminology Association Seminar 944E0470A Moscow DELOVOY MIR in Russian No 14, 24-30 Jan 94 p 20 944E0470A Moscow DELOVOY MIR Language: Russian Article Type:CSO [Article by Yuriy Nekrasov, member of the Criminology Association, under the heading ""Busi...",,1,4.230750,international organized crime,applypipeline:off criminal^0.0017062496743867252 affairs^0.0011665247657870739 organs^0.0008983014181774731 internal^0.0008755314680525699 russian^0.000789959973314172 economic^0.0007703573750240684 groups^0.0006595210159383628 counterintelligence^0.0006267200155651165 measures^0.000556643955467908 banditry^0.0005554692128284883 activity^0.000522475248569999 serious^0.0005198452845253485 investigations^0.0004935254982725303 report^0.00048782945361054144 shall^0.0004792792344895164 financial^...
2,301,42623,FBIS3-42624,"""jpusr009___94024"" FBIS-USR-94-009 Document Type:JPRS Document Title: FBIS Report: Central Eurasia 3 February 1994 RUSSIA ECONOMIC AND SOCIAL AFFAIRS DELOVOY MIR Survey of Enterprise Directors, 11 January 944E0374A Moscow DELOVOY MIR in Russian 11 Jan 94 p 3 944E0374A Moscow DELOVOY MIR Language: Russian Article Type:CSO [Article by Candidate of Economic Sciences Vladimir Belyanov: ""There Is No Upsurge in Sight: Enterprise Directors Appraise t...",,2,4.082481,international organized crime,applypipeline:off criminal^0.0017062496743867252 affairs^0.0011665247657870739 organs^0.0008983014181774731 internal^0.0008755314680525699 russian^0.000789959973314172 economic^0.0007703573750240684 groups^0.0006595210159383628 counterintelligence^0.0006267200155651165 measures^0.000556643955467908 banditry^0.0005554692128284883 activity^0.000522475248569999 serious^0.0005198452845253485 investigations^0.0004935254982725303 report^0.00048782945361054144 shall^0.0004792792344895164 financial^...
3,301,23975,FBIS3-23976,"""jpusr023___94039"" FBIS-USR-94-023 Document Type:JPRS Document Title: FBIS Report: Central Eurasia 10 March 1994 RUSSIA ECONOMIC AND SOCIAL AFFAIRS Agricultural Newspaper Reports Weather, Crop Conditions Effect of Thaws Noted 944K0784C Moscow SELSKAYA ZHIZN in Russian 1 Mar 94 p 5 944K0784C Moscow SELSKAYA ZHIZN Language: Russian Article Type:CSO [Article by health resort physician Asya Uyanayeva under the rubric ""Meteorological Dictionary...",,3,3.885892,international organized crime,applypipeline:off criminal^0.0017062496743867252 affairs^0.0011665247657870739 organs^0.0008983014181774731 internal^0.0008755314680525699 russian^0.000789959973314172 economic^0.0007703573750240684 groups^0.0006595210159383628 counterintelligence^0.0006267200155651165 measures^0.000556643955467908 banditry^0.0005554692128284883 activity^0.000522475248569999 serious^0.

In [117]:
(BM25>>RM3>>BM25)(topicsRobustTT.head(1))

BR(BM25):   0%|          | 0/1 [00:00<?, ?q/s]

BR(BM25):   0%|          | 0/1 [00:00<?, ?q/s]

,qid,docid,docno,body,title,rank,score,query_0,query
0,301,103568,FBIS4-41991,"""drsov126_a_94023"" FBIS-SOV-94-126 Dail...",,0,24.154745,international organized crime,applypipeline:off intern^0.200000018 russian^0...
1,301,101837,FBIS4-40260,"""drsov115_b_94002"" FBIS-SOV-94-115 Dail...",,1,23.191582,international organized crime,applypipeline:off intern^0.200000018 russian^0...
2,301,23985,FBIS3-23986,"""jpusr023___94049"" FBIS-USR-94-023 Docu...",,2,22.817223,international organized crime,applypipeline:off intern^0.200000018 russian^0...
3,301,103416,FBIS4-41839,"""drsov125_b_94003"" FBIS-SOV-94-125 Dail...",,3,22.575516,international organized crime,applypipeline:off intern^0.200000018 russian^0...
4,301,99941,FBIS4-38364,"""drsov104_a_94008"" FBIS-SOV-94-104 Dail...",,4,21.938001,international organized crime,applypipeline:off intern^0.200000018 russian^0...
...,...,...,...,...,...,...,...,...,...
995,301,410508,LA021189-0023,"17012 February 11, 1989, Saturday, Home Edi...",,995,12.447238,international organized crime,applypipeline:off intern^0.200000018 russian^0...
996,301,105274,FBIS4-43697,"""drweu119__a94048"" FBIS-WEU-94-119-A Da...",,996,12.446622,international organized crime,applypipeline:off intern^0.200000018 russian^0...
997,301,51765,FBIS3-51766,"""drlat016__a94044"" FBIS-LAT-94-016-A Do...",,997,12.444559,international organized crime,applypipeline:off intern^0.200000018 russian^0...
998,301,122847,FBIS4-61270,"""drsov087_s_94005"" FBIS-SOV-94-087 Dail...",,998,12.442529,international organized crime,applypipeline:off intern^0.200000018 russian^0...


# Experiment

In [134]:
ceqe_pipe = BM25>>ceqe( num_fb_docs=22, num_exp_terms=71, method ="centroid",verbose=True)\
 >>pt.apply.query_0(lambda row: row["query"])\
 >>pt.apply.query(lambda row: "applypipeline:off "+row["ceqe_expanded_query"]) \
 >>pt.apply.query_1(drop=True)\
 >>pt.apply.ceqe_expanded_query(drop=True)>>pt.apply.docno(drop=True)>>BM25
 

In [135]:
from pyterrier.measures import *
pt.Experiment(
    [BM25>>RM3>>BM25,
    ceqe_pipe],
    topicsRobustTT.head(5),
    qrelsRobust,
    eval_metrics=["map","ndcg_cut_10","ndcg_cut_20","recall.100","recall","recip_rank", "mrt"],
#     drop_unused = True,
    names=["baseline","ceqe"]
)

BR(BM25):   0%|          | 0/5 [00:00<?, ?q/s]

BR(BM25):   0%|          | 0/5 [00:00<?, ?q/s]

BR(BM25):   0%|          | 0/5 [00:00<?, ?q/s]

/tmp/ipykernel_7400/3961208512.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  embedding_matrix_non_zero = np.array(embedding_matrix_non_zero)
/users/tr.xiaow/ceqe-release/main/tokens_to_terms.py:56: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(queries_terms_embeds_pooled)


BR(BM25):   0%|          | 0/5 [00:00<?, ?q/s]

23:47:35.701 [main] WARN org.terrier.matching.PostingListManager - A negative requirement was found for the 606-th query term (05), which was past the maximum supported 64
23:47:35.803 [main] WARN org.terrier.matching.PostingListManager - A negative requirement was found for the 2366-th query term (06), which was past the maximum supported 64
23:47:39.625 [main] WARN org.terrier.matching.PostingListManager - A negative requirement was found for the 98-th query term (14), which was past the maximum supported 64
23:47:39.641 [main] WARN org.terrier.matching.PostingListManager - A negative requirement was found for the 253-th query term (12), which was past the maximum supported 64
23:47:39.654 [main] WARN org.terrier.matching.PostingListManager - A negative requirement was found for the 295-th query term (13), which was past the maximum supported 64
23:47:39.666 [main] WARN org.terrier.matching.PostingListManager - A negative requirement was found for the 347-th query term (07), which wa

,name,map,ndcg_cut_10,ndcg_cut_20,recall.100,R@5,R@10,R@15,R@20,R@30,R@200,R@500,R@1000,recip_rank,mrt
0,baseline,0.168305,0.252384,0.258352,0.404266,0.035241,0.044472,0.056190,0.069965,0.100151,0.434526,0.480970,0.540932,0.424074,931.770529
1,ceqe,0.004383,0.118521,0.104840,0.053829,0.002360,0.002360,0.003253,0.006776,0.006776,0.059968,0.088896,0.094636,0.319591,14757.951184


In [147]:
res = BM25(topicsRobustTT)
res

BR(BM25):   0%|          | 0/250 [00:00<?, ?q/s]

,qid,docid,docno,body,title,rank,score,query
0,301,101837,FBIS4-40260,"""drsov115_b_94002"" FBIS-SOV-94-115 Daily Report 15 Jun 1994 RUSSIA NATIONAL AFFAIRS Political Issues Text of Presidential Edict on Organized Crime Text of Presidential Edict on Organized Crime PM1506102194 Moscow IZVESTIYA in Russian 15 Jun 94 p 1 PM1506102194 Moscow IZVESTIYA Russian BFN [Unattributed report: ""President Takes Emergency Measures in Fight against Crime Orgy""] [Text] On 14 June, President B. Yeltsin signed the edict ""On...",,0,21.679224,international organized crime
1,301,23985,FBIS3-23986,"""jpusr023___94049"" FBIS-USR-94-023 Document Type:JPRS Document Title: FBIS Report: Central Eurasia 10 March 1994 RUSSIA ECONOMIC AND SOCIAL AFFAIRS Seminar on Criminology Held 1990-1993 Crime Figures, Pessimistic Prognosis for Future 944E0470B Moscow DELOVOY MIR in Russian No 14, 24-30 Jan 94 p 20 944E0470B Moscow DELOVOY MIR Language: Russian Article Type:CSO [Unattributed report: ""The Forecast Is For Foul Weather (From the Theses Prepare...",,1,21.149702,international organized crime
2,301,103568,FBIS4-41991,"""drsov126_a_94023"" FBIS-SOV-94-126 Daily Report 29 Jun 1994 RUSSIA INTERNATIONAL AFFAIRS Freeh Visit To Focus on Organized Crime Freeh Visit To Focus on Organized Crime PM2906155694 Moscow IZVESTIYA in Russian 29 Jun 94 p 3 PM2906155694 Moscow IZVESTIYA Russian BFN [Report by Vladimir Nadein: ""FBI Is Ready To Take on Russian Mafia""] [Text] Washington -- FBI Director Louis Freeh will spend the coming week in a number of Central and East E...",,2,21.136257,international organized crime
3,301,69388,FBIS4-7811,"""dreeu067_m_94002"" FBIS-EEU-94-067 Daily Report 12 Mar 1994 THE FORMER YUGOSLAV REPUBLIC OF MACEDONIA Government Ties to Organized Crime Examined Government Ties to Organized Crime Examined 94BA0230A Skopje NOVA MAKEDONIJA in Macedonian 12 Mar 94 p 16 94BA0230A Skopje NOVA MAKEDONIJA Macedonian CSO [Article by Zoran Petrov: ""The Thieves Are Not Among Us--We Are Among Them""] [Text] Now there are already numerous indications, information, ...",,3,21.011514,international organized crime
4,301,108311,FBIS4-46734,"""jpusr059___94025"" FBIS-USR-94-059 JPRS FBIS Report: Central Eurasia 19 April 1994 RUSSIA REGIONAL AFFAIRS North Caucasus Anticrime Chief Views Current Tasks North Caucasus Anticrime Chief Views Current Tasks 944F0736A Nalchik KABARDINO-BALKARSKAYA PRAVDA in Russian 19 Apr 94 p 3 944F0736A Nalchik KABARDINO-BALKARSKAYA PRAVDA Russian CSO [Interview with Col Militia Ruslan Shkhamgeriyevich Yeshugaov, chief of the Administration for ...",,4,20.726418,international organized crime
...,...,...,...,...,...,...,...,...
242403,700,147678,FR940505-1-00225,"FR940505-1-00012 EPA requests comment on the feasibility of meeting an NO X standard of 1.5 g/bhp-hr for all engine applications, as well as on the need for the surcharge-based 2.5 g/bhp-hr standard.EPA also requests comment on the possibility of adopting an NO X standard of 2.5 g/bhp-hr for all California on-highway heavy-duty engines without a new vehicle emissions surcharge. The Agency also notes that in the longer term, significant potential exists for low NO X emissions...",,995,12.276608,gasoline tax u s
242404,700,440453,LA050489-0037,"52261 May 4, 1989, Thursday, Home Edition Part 1; Page 1; Column 1; Metro Desk 1492 words CLEAN-FUEL PROGRAM GOALS NOW APPEAR UNATTAINABLE By LARRY B. STAMMER and MAURA DOLAN, Times Environmental Writers Evidence is mounting that regional air quality officials have set unattainable goals for replacing millions of gasoline- and diesel-fueled cars, trucks and buses over the next 10 years with vehicles that run on cleaner fuels. The clean-fuels program is a cornerstone ...",,996,12.265247,gasoline tax u s
242405,700,502670,LA102490-0004,"296821 October 24, 1990, Wednesday, Ventura County Edition Metro; Part B; Page 2; Column 5 237 words VENTURA COUNTY NEWS ROUNDUP: BUS LINE TO BE RUN ON NATURAL GAS By MAJA RADEVICH A bus aimed at reducing pollu

In [146]:
from pyterrier.measures import *
pt.Experiment(
    [
#         BM25>>RM3>>BM25,
    ceqe_pipe],
    topicsRobustTT,
    qrelsRobust,
    eval_metrics=["map","ndcg_cut_10","ndcg_cut_20","recall.100","recall","recip_rank", "mrt"],
#     drop_unused = True,
    names=["ceqe-centroid"]
)

BR(BM25):   0%|          | 0/250 [00:00<?, ?q/s]

/tmp/ipykernel_7400/3961208512.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  embedding_matrix_non_zero = np.array(embedding_matrix_non_zero)


IndexError: single positional indexer is out-of-bounds

In [141]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25", controls={"c" : 0.5},verbose=True)
rm3 = pt.rewrite.RM3(index, fb_terms=72, fb_docs=22)

In [144]:
# bm25>>rm3>>bm25

In [143]:
from pyterrier.measures import *
pt.Experiment(
    [bm25>>rm3>>bm25,
    ],
    topicsRobustTT,
    qrelsRobust,
    eval_metrics=["map","ndcg_cut_10","ndcg_cut_20","recall.100","recall","recip_rank", "mrt"],
#     drop_unused = True,
    names=["baseline"]
)

BR(BM25):   0%|          | 0/250 [00:00<?, ?q/s]

BR(BM25):   0%|          | 0/250 [00:00<?, ?q/s]

,name,map,ndcg_cut_10,ndcg_cut_20,recall.100,R@5,R@10,R@15,R@20,R@30,R@200,R@500,R@1000,recip_rank,mrt
0,baseline,0.278417,0.447114,0.42762,0.440877,0.088205,0.144486,0.187548,0.215157,0.264285,0.544682,0.673996,0.752341,0.666286,935.439971
